#### Notebook for running React experiments

In [1]:
import sys, os
sys.path.append('..')
root  = '../root/'

In [1]:
import os 

os.environ['OPENAI_API_KEY'] = ""
print(os.environ['OPENAI_API_KEY'])

In [4]:
import joblib
from util import summarize_react_trial, log_react_trial, save_agents
from agents import ReactReflectAgent, ReactAgent, ReflexionStrategy

#### Load the HotpotQA Sample

In [5]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

FileNotFoundError: [Errno 2] No such file or directory: '../data/hotpot-qa-distractor-sample.joblib'

#### Define the Reflexion Strategy

In [29]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [40]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Initialize a React Agent for each question

In [41]:
agent_cls = ReactReflectAgent if strategy != ReflexionStrategy.NONE else ReactAgent
agents = [agent_cls(row['question'], row['answer']) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [42]:
n = 5
trial = 0
log = ''

In [33]:
len(agents)

100

In [43]:
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        if strategy != ReflexionStrategy.NONE:
            agent.run(reflect_strategy = strategy)
        else:
            agent.run()
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_react_trial(agents, trial)
    correct, incorrect, halted = summarize_react_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}, Halted: {len(halted)}')

Thought 1: I need to search VIVA Media AG and find out their new acronym.
Action 1: Search[VIVA Media AG]
HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=VIVA+Media+AG&srinfo=suggestion&format=json&action=query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f817dbea370>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
Observation 1: Could not find that page, please try again.
Thought 2: I couldn't find the page for VIVA Media AG. I should try searching for similar entities.
Action 2: Search[similar entities to VIVA Media AG]
HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=similar+entities+to+VIVA+Media+AG&srinfo=suggestion&format=json&action=query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f817de32dc0>: F

#### Save the result log

In [ ]:
with open(os.path.join(root, 'ReAct', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join('ReAct', strategy.value, 'agents'))